In [1]:
!pip install openai transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import openai
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import random
import re

In [3]:
class StoryGenerator:
    def __init__(self):
        print("Loading GPT-2 model...")
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.model = GPT2LMHeadModel.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token
        print("Model loaded successfully!")

    def generate_story_gpt2(self, prompt, max_length=200, temperature=0.8):
        # Encode the prompt
        inputs = self.tokenizer.encode(prompt, return_tensors="pt")

        # Generate story
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                temperature=temperature,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                no_repeat_ngram_size=2
            )

        # Decode and clean the story
        story = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self.clean_story(story)

    def clean_story(self, story):
        # Clean up the generated story
        sentences = story.split('.')
        cleaned_sentences = []

        for sentence in sentences[:8]:  # Keep first 8 sentences max
            sentence = sentence.strip()
            if len(sentence) > 10 and not sentence.startswith(' '):
                cleaned_sentences.append(sentence)

        return '. '.join(cleaned_sentences) + '.'

# Initialize the story generator
story_gen = StoryGenerator()

Loading GPT-2 model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!


In [4]:
class OpenAIStoryGenerator:
    def __init__(self, api_key=None):
        if api_key:
            openai.api_key = api_key
            self.has_api = True
        else:
            self.has_api = False

    def generate_story_openai(self, prompt, max_tokens=300):
        if not self.has_api:
            return "OpenAI API key not provided. Please set your API key to use this feature."

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a creative story writer. Write engaging short stories."},
                    {"role": "user", "content": f"Write a short story based on this prompt: {prompt}"}
                ],
                max_tokens=max_tokens,
                temperature=0.8
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating story: {str(e)}"

# Uncomment and add your OpenAI API key if you have one
openai_gen = OpenAIStoryGenerator("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")


In [5]:
class PromptGenerator:
    def __init__(self):
        self.characters = [
            "a brave knight", "a young wizard", "a mysterious stranger",
            "an old librarian", "a space explorer", "a detective",
            "a talking cat", "a robot", "a pirate captain", "a time traveler"
        ]

        self.settings = [
            "in a haunted castle", "on a distant planet", "in a bustling city",
            "in an enchanted forest", "aboard a spaceship", "in ancient Egypt",
            "in a steampunk world", "underwater", "in the clouds", "in a cyberpunk future"
        ]

        self.conflicts = [
            "must solve a mystery", "discovers a hidden treasure", "faces their greatest fear",
            "must save the world", "finds a magical artifact", "meets their doppelganger",
            "gets lost in time", "must break a curse", "discovers a conspiracy", "faces an impossible choice"
        ]

    def generate_random_prompt(self):
        character = random.choice(self.characters)
        setting = random.choice(self.settings)
        conflict = random.choice(self.conflicts)

        prompt = f"Once upon a time, {character} {setting} {conflict}."
        return prompt

    def get_genre_prompt(self, genre):
        genre_prompts = {
            "fantasy": "In a magical realm where dragons soar through crystal skies,",
            "sci-fi": "In the year 2157, when humans have colonized Mars,",
            "mystery": "The old mansion held secrets that hadn't been revealed for decades,",
            "adventure": "The treasure map led to the most dangerous place on Earth,",
            "horror": "As the clock struck midnight, strange shadows began to move,",
            "romance": "Two strangers met on a rainy day at a small coffee shop,"
        }
        return genre_prompts.get(genre, "Once upon a time,")

prompt_gen = PromptGenerator()

In [6]:
def generate_story_interactive():
    print("=== AI Story Generator ===\n")

    # Get user preferences
    print("Choose your story generation method:")
    print("1. Random prompt")
    print("2. Genre-based prompt")
    print("3. Custom prompt")

    choice = input("\nEnter your choice (1-3): ").strip()

    if choice == "1":
        prompt = prompt_gen.generate_random_prompt()
        print(f"\nGenerated prompt: {prompt}")

    elif choice == "2":
        print("\nAvailable genres: fantasy, sci-fi, mystery, adventure, horror, romance")
        genre = input("Enter genre: ").strip().lower()
        prompt = prompt_gen.get_genre_prompt(genre)
        print(f"\nGenre prompt: {prompt}")

    elif choice == "3":
        prompt = input("\nEnter your custom prompt: ").strip()

    else:
        prompt = "Once upon a time, in a land far away,"
        print(f"\nUsing default prompt: {prompt}")

    # Generate story
    print("\nGenerating story... Please wait...")

    # You can choose between GPT-2 (offline) or OpenAI (requires API key)
    story = story_gen.generate_story_gpt2(prompt, max_length=250, temperature=0.7)

    print("\n" + "="*60)
    print(" YOUR AI-GENERATED STORY:")
    print("="*60)
    print(story)
    print("="*60)

    return story

# Run the interactive story generator
generate_story_interactive()

# Cell 7: Batch Story Generation and Save
def generate_multiple_stories(num_stories=3):
    print(f"Generating {num_stories} random stories...\n")

    stories = []
    for i in range(num_stories):
        prompt = prompt_gen.generate_random_prompt()
        print(f"Story {i+1} - Prompt: {prompt}")

        story = story_gen.generate_story_gpt2(prompt, max_length=200)
        stories.append({
            'prompt': prompt,
            'story': story
        })

        print(f"Story {i+1}:")
        print(story)
        print("\n" + "-"*50 + "\n")

    # Save stories to file
    with open('ai_generated_stories.txt', 'w') as f:
        for i, story_data in enumerate(stories, 1):
            f.write(f"Story {i}\n")
            f.write(f"Prompt: {story_data['prompt']}\n")
            f.write(f"Story: {story_data['story']}\n")
            f.write("\n" + "="*50 + "\n\n")

    print(f" {num_stories} stories saved to 'ai_generated_stories.txt'")
    return stories

# Generate multiple stories
stories = generate_multiple_stories(3)

# Bonus: Story statistics
def analyze_stories(stories):
    total_words = sum(len(story['story'].split()) for story in stories)
    avg_words = total_words / len(stories)

    print(f"\n Story Statistics:")
    print(f"Total stories generated: {len(stories)}")
    print(f"Total words: {total_words}")
    print(f"Average words per story: {avg_words:.1f}")

analyze_stories(stories)

=== AI Story Generator ===

Choose your story generation method:
1. Random prompt
2. Genre-based prompt
3. Custom prompt

Enter your choice (1-3): 1


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Generated prompt: Once upon a time, a robot in a bustling city must solve a mystery.

Generating story... Please wait...

 YOUR AI-GENERATED STORY:
Once upon a time, a robot in a bustling city must solve a mystery. In my mind, an "old man" was the most interesting man in the place. And in our minds, he was our "new man". "The man who was, I think, quite a pleasant man.
Generating 3 random stories...

Story 1 - Prompt: Once upon a time, a young wizard in a haunted castle faces an impossible choice.
Story 1:
Once upon a time, a young wizard in a haunted castle faces an impossible choice. A battle with ghosts is not an easy one; so many questions lay ahead in the maze of a world of dark nightmares. The first two stories are an original story-telling tale and a fantasy mystery. The third one is a story of magic involving the mind and body. It's an allusion to ancient legend and lore of the undead and how that mythology may have played a role in our own time.

-----------------------------